# Strutture dati ed algoritmi

## Alberi

La rappresentazione più comune che sarà adoperata per il corso per gli alberi $n$-ari sono le *lol* (liste di liste)

In [ ]:
import os
os.environ["ANTLR4_JAR"] = "/home/federicobruzzoneplasma/Documents/FedericoBruzzone/master-courses/linguaggi-e-traduttori/lecture/jars/antlr-4.12.0-complete.jar"


In [ ]:
# [radice] 
# [radice alberi…]

tree = [1, [11], [12, [121], [122]], [13]]

Accedere a radice e figli con l'[iterable unpacking](https://docs.python.org/3/reference/expressions.html?highlight=iterable+unpacking#expression-lists)…

In [ ]:
root, *children = tree

In [ ]:
# uso di liblet per ottenre una rappresentazione grafica 

from liblet import Tree

t = Tree.from_lol(tree)
t

### Visite

* preordine, 
* postordine, 
* per livello.

In [ ]:
def preorder(tree, visitor):
  root, *children = tree
  visitor(root)
  for st in children: preorder(st, visitor)

t

In [ ]:
preorder(tree, print)

In [ ]:
def postorder(tree, visitor):
  root, *children = tree
  for st in children: postorder(st, visitor)
  visitor(root)

t

In [ ]:
postorder(tree, print)

In [ ]:
from liblet import Queue

def levelorder(tree, visitor):

  Q = Queue()

  Q.enqueue(tree)
  while Q:
    tree = Q.dequeue()
    root, *children = tree
    visitor(root)
    for child in children: Q.enqueue(child)

t

In [ ]:
levelorder(tree, print)

### Alberi con attributi 

Per ora gli alberi avevano interi come velori dei nodi, costruiamo un albero che abbia `dict` come valori (e che conservi il valore numerico come valore della chiave `val`).

In [ ]:
def add_attr(tree):
  root, *children = tree
  return [{'val': root}] + [add_attr(child) for child in children]

In [ ]:
tree = [1, [11], [12, [121], [122]], [13]]

add_attr(tree)

In [ ]:
Tree.from_lol(add_attr(tree))

#### Attributi ereditati e preorder

Come vedremo più avanti, gli attributi ereditati sono attributi che i nodi dei sottoalberi ereditano dal padre; ad esempio la *profondità*.

In [ ]:
def add_depth(tree, parent):
  root, *children = tree
  root['depth'] = parent['depth'] + 1
  for tree in children: add_depth(tree, root)

In [ ]:
attr_tree = add_attr(tree)

# uso il nodo fittizio {'depth': 0} come "primo" parent dell'albero

add_depth(attr_tree, {'depth': 0}) 

Tree.from_lol(attr_tree)

#### Attributi sintetizzati e postorder

Gli attributi sintetizzati sono attributi che il nodo radice di un albero ricava dal valore degli attributi nei sottoalberi; ad esempio, il *massimo* valore.

In [ ]:
def add_max(tree):
  root, *children = tree
  if not children: # il massimo di una foglia è il suo valore
    root['max'] = root['val']
  else:
    for child in children: add_max(child)
    root['max'] = max(child[0]['max'] for child in children)

In [ ]:
attr_tree = add_attr(tree)

add_max(attr_tree) 

Tree.from_lol(attr_tree)

## Grafi

Per i grafi sono usuali due rappresentazioni: per *archi* (dappresentati da `tuple` di `tuple`) e tramite la relazione di *adiacenza* (rappresentata da un `dict` di `set`).

In [ ]:
arcs = (
  (1, 2), 
  (1, 4),
  (2, 3), 
  (3, 2), 
  (3, 4), 
  (3, 5)
)

In [ ]:
from liblet import Graph

g = Graph(arcs)
g

In [ ]:
# dagli archi alla mappa delle adiacenze


# per ogni nodo n (sia s o t), adjacency[n] = set()

adjacency = dict()
for s, t in arcs:
  adjacency[s] = set()
  adjacency[t] = set()

# aggiungo gli outlink

for s, t in arcs: adjacency[s] |= {t}

adjacency

In [ ]:
# e viceversa 

for s, ts in adjacency.items():
  for t in ts: print(s, t)

### Visite

* ampiezza,
* profondità.

In [ ]:
def depthfirst(adjacency, start, visit):
  def walk(src):
    visit(src)
    seen.add(src)
    for dst in adjacency[src]:
      if dst not in seen: 
        walk(dst)
  seen = set()
  walk(start)

g

In [ ]:
depthfirst(adjacency, 1, print)

In [ ]:
def breadthfirst(adjacency, start, visit):

  Q = Queue()

  seen = set()
  Q.enqueue(start)
  while Q:
    src = Q.dequeue()
    visit(src)
    seen.add(src)
    for dst in adjacency[src]:
      if dst not in seen:
        Q.enqueue(dst)

In [ ]:
breadthfirst(adjacency, 1, print)

## Backtracking

Il [backtracking](https://en.wikipedia.org/wiki/Backtracking) è uno schema di algoritmi ricorsivi per problemi la cui soluzione possa essere costruita incrementalmente a partire da una soluzione "candidata". Lo schema generale è

```python
def backtrack(candidate):
    if reject(candidate): return
    if accept(candidate): output(candidate)
    s = first(candidate)
    while s:
        backtrack(s)
        s = next(candidate)
```

Le funzioni `reject` e `accept` hanno l'ovvio significato di indicare, rispettivamente, se una soluzione candidata è non corretta (e non ulteriormente emendabile), oppure se costituisce una soluzione (completa). Le funzioni `first` e `next` costruiscono rispettivamente il primo e i successivi candidati a partire dal candidato corrente.

### Segmentazione di una parola

In [ ]:
from urllib.request import urlopen

with urlopen('https://raw.githubusercontent.com/napolux/paroleitaliane/master/paroleitaliane/60000_parole_italiane.txt') as url: 
  WORDS = {word.decode().strip().upper() for word in url if len(word) >= 3}

print(len(WORDS))

In [ ]:
def segmenta(segmenti, resto):
  if segmenti and not segmenti[-1] in WORDS: return
  if not resto: 
    print(segmenti)
    return
  for i in range(1, 1 + len(resto)):
    segmenta(segmenti + [resto[:i]], resto[i:])

In [ ]:
segmenta([], 'ILCORRIEREDELLASERAEDIZIONENOTTURNA')

#### Esempi supplementari

* calcolare la [densità delle soluzioni](https://gist.github.com/mapio/2c8b171110dc6a09dfd6) del [problema delle otto regine](https://en.wikipedia.org/wiki/Eight_queens_puzzle),

* enumerare le [soluzioni](https://gist.github.com/mapio/967f3d8793fcab80941dc0b4f370dbeb) del gioco [Find a way](https://play.google.com/store/apps/details?id=com.zerologicgames.findaway),

* trovare le [soluzioni](https://gist.github.com/mapio/33f1c381870333fe502e) del [Sudoku](https://en.wikipedia.org/wiki/Sudoku).